In [5]:
import subprocess

def get_macos_theme():
    """
    Determine the display theme for a macOS computer using AppleScript.

    Returns:
    :return str: 'light', 'dark', or 'none' based on the current display theme.

    Example:
    >>> get_macos_theme()
    'dark'
    """
    try:
        # AppleScript to get the appearance setting
        script = '''
        tell application "System Events"
            tell appearance preferences
                if (dark mode) then
                    return "dark"
                else
                    return "light"
                end if
            end tell
        end tell
        '''
        # Run the AppleScript
        result = subprocess.run(
            ['osascript', '-e', script],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Check the output
        if result.returncode == 0:
            theme = result.stdout.strip().lower()
            if theme in ['dark', 'light']:
                return theme
            else:
                return 'none'
        else:
            return 'none'
    except Exception as e:
        # In case of any exception, return 'none'
        return 'none'

# Example usage
print(get_macos_theme())

dark


In [1]:
def get_nth_quadrant(self, angle_deg, base):
    """
    Return the quadrant in which an angle in degrees lies if the area is described by dividing a unit circle into 'base' equal segments.

    :param angle_deg (float): The angle in degrees.
    :param base (int): The number of segments to divide the circle into (must be even and up to 16).

    Returns:
    :return str: The quadrant (0-F) that the angle belongs to.
    """
    if base < 2 or base > 16 or base % 2 != 0:
        raise ValueError("Base must be an even number between 2 and 16.")

    # Normalize the angle to the range [0, 360)
    angle_deg = angle_deg % 360

    # Calculate the size of each segment
    segment_size = 360 / base

    # Determine the quadrant
    for i in range(base):
        if angle_deg >= i * segment_size and angle_deg < (i + 1) * segment_size:
            return format(base - 1 - i, "X")

    # If no quadrant is found, raise an error (this should not happen)
    raise ValueError("Invalid angle value: angle must be in the range [0, 360).")